# Customer churn prediction
Customer churn prediction is to measure why customers are leaving a business.

1. Build a deep learning model to predict churn rate.
2. Improve f1 score in minority class using various techniques such as undersampling, oversampling, ensemble, etc.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('customer_churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.drop('customerID', axis=1, inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  PaperlessBilling  7043 non-null   object 
 16  PaymentMethod     7043 non-null   object 


**TotalCharges should be float but it is an object. Let's check what's going on with this column**

In [5]:
df['TotalCharges'].values

array(['29.85', '1889.5', '108.15', ..., '346.45', '306.6', '6844.5'],
      dtype=object)

**Its string. Convert it to numbers.**

In [6]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
# 'coerce' argument will convert non-numeric values to NaN

In [7]:
df['TotalCharges'].isnull().sum()

11

Remove rows with NaN values in 'TotalCharges' column

In [8]:
df = df.dropna(subset=['TotalCharges'])

**Print unique values in object columns to see data values**

In [9]:
def print_unique_col_values(df):
    for col in df:
        if df[col].dtypes == 'object':
            print (f'{col}: {df[col].unique()}')

In [10]:
print_unique_col_values(df)

gender: ['Female' 'Male']
Partner: ['Yes' 'No']
Dependents: ['No' 'Yes']
PhoneService: ['No' 'Yes']
MultipleLines: ['No phone service' 'No' 'Yes']
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: ['No' 'Yes' 'No internet service']
OnlineBackup: ['Yes' 'No' 'No internet service']
DeviceProtection: ['No' 'Yes' 'No internet service']
TechSupport: ['No' 'Yes' 'No internet service']
StreamingTV: ['No' 'Yes' 'No internet service']
StreamingMovies: ['No' 'Yes' 'No internet service']
Contract: ['Month-to-month' 'One year' 'Two year']
PaperlessBilling: ['Yes' 'No']
PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
Churn: ['No' 'Yes']


**Some of the columns have no internet service or no phone service, that can be replaced with a simple No**

In [11]:
df1 = df.copy()

df1.replace('No internet service','No',inplace=True)
df1.replace('No phone service','No',inplace=True)

**Convert binary categorical values to numeric values**

In [12]:
df1['gender'] = df['gender'].replace({'Female': 0, 'Male': 1})

for col in df1.columns:
    df1[col] = df1[col].replace({'Yes': 1, 'No': 0})
    
for col in df1.columns:
    print(f'{col}: {df1[col].unique()}')

gender: [0 1]
SeniorCitizen: [0 1]
Partner: [1 0]
Dependents: [0 1]
tenure: [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
PhoneService: [0 1]
MultipleLines: [0 1]
InternetService: ['DSL' 'Fiber optic' 0]
OnlineSecurity: [0 1]
OnlineBackup: [1 0]
DeviceProtection: [0 1]
TechSupport: [0 1]
StreamingTV: [0 1]
StreamingMovies: [0 1]
Contract: ['Month-to-month' 'One year' 'Two year']
PaperlessBilling: [1 0]
PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
MonthlyCharges: [29.85 56.95 53.85 ... 63.1  44.2  78.7 ]
TotalCharges: [  29.85 1889.5   108.15 ...  346.45  306.6  6844.5 ]
Churn: [0 1]


**One hot encoding for categorical columns**

### One hot encoding

In [13]:
print_unique_col_values(df1)

InternetService: ['DSL' 'Fiber optic' 0]
Contract: ['Month-to-month' 'One year' 'Two year']
PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']


In [14]:
categorical_cols = ['InternetService', 'Contract', 'PaymentMethod']
df2 = pd.get_dummies(df1, columns=categorical_cols, drop_first=True).astype(int)

df2.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'PaperlessBilling', 'MonthlyCharges', 'TotalCharges', 'Churn',
       'InternetService_DSL', 'InternetService_Fiber optic',
       'Contract_One year', 'Contract_Two year',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check'],
      dtype='object')

In [15]:
df2.sample(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,MonthlyCharges,TotalCharges,Churn,InternetService_DSL,InternetService_Fiber optic,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
1239,0,0,1,1,3,1,0,0,1,0,...,58,168,0,1,0,0,0,1,0,0
1757,1,0,1,1,54,1,0,0,1,0,...,79,4362,0,0,1,0,1,0,0,0
4752,1,0,0,0,23,1,0,0,0,0,...,20,481,0,0,0,1,0,0,0,1
5604,1,0,0,0,17,1,0,0,0,0,...,70,1207,0,0,1,0,0,0,1,0
6530,1,1,1,0,63,0,0,1,0,0,...,36,2298,0,1,0,1,0,0,0,0


**Scaling values**

In [16]:
cols_to_scale = ['tenure', 'MonthlyCharges', 'TotalCharges']
df2[cols_to_scale].describe()

,tenure,MonthlyCharges,TotalCharges
count,7032.000000,7032.000000,7032.000000
mean,32.421786,64.332480,2282.830489
std,24.545260,30.088668,2266.768490
min,1.000000,18.000000,18.000000
25%,9.000000,35.000000,401.000000
50%,29.000000,70.000000,1397.000000
75%,55.000000,89.000000,3794.250000
max,72.000000,118.000000,8684.000000


In [17]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df2[cols_to_scale] = scaler.fit_transform(df2[cols_to_scale])

In [18]:
df2[cols_to_scale].describe()

,tenure,MonthlyCharges,TotalCharges
count,7032.000000,7032.000000,7032.000000
mean,0.442560,0.463325,0.261347
std,0.345708,0.300887,0.261570
min,0.000000,0.000000,0.000000
25%,0.112676,0.170000,0.044196
50%,0.394366,0.520000,0.159128
75%,0.760563,0.710000,0.435755
max,1.000000,1.000000,1.000000


## Model

In [19]:
from sklearn.model_selection import train_test_split

X = df2.drop('Churn', axis=1)
y = df2['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# stratify=y ensures that the distribution of classes in the target variable is maintained in both the training and testing sets

In [20]:
print(y.value_counts())
print(y_train.value_counts())
print(y_test.value_counts())

Churn
0    5163
1    1869
Name: count, dtype: int64
Churn
0    4130
1    1495
Name: count, dtype: int64
Churn
0    1033
1     374
Name: count, dtype: int64


In [21]:
5163/1869, 4130/1495, 1033/374

(2.7624398073836276, 2.762541806020067, 2.7620320855614975)

In [22]:
X_train.shape, X_test.shape

((5625, 23), (1407, 23))

## ANN

**Build a model (ANN) in tensorflow/keras**

In [23]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report

In [24]:
def ANN(X_train, X_test, y_train, y_test):
    model = keras.Sequential([
        keras.layers.Dense(23, input_shape=(23,), activation='relu'),
        keras.layers.Dense(12, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    model.fit(X_train, y_train, epochs=50, verbose=2)

    print('\nModel Evaluation:\n')
    print(model.evaluate(X_test, y_test))

    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred)

    # print('\nClassification Report:\n')
    # print(classification_report(y_test, y_pred))

    return y_pred

In [25]:
y_pred = ANN(X_train, X_test, y_train, y_test)

Epoch 1/50
176/176 - 4s - loss: 0.5327 - accuracy: 0.7397 - 4s/epoch - 20ms/step
Epoch 2/50
176/176 - 1s - loss: 0.4344 - accuracy: 0.7934 - 1s/epoch - 7ms/step
Epoch 3/50
176/176 - 1s - loss: 0.4229 - accuracy: 0.7996 - 1s/epoch - 7ms/step
Epoch 4/50
176/176 - 1s - loss: 0.4170 - accuracy: 0.8021 - 1s/epoch - 8ms/step
Epoch 5/50
176/176 - 1s - loss: 0.4157 - accuracy: 0.7995 - 1s/epoch - 6ms/step
Epoch 6/50
176/176 - 1s - loss: 0.4135 - accuracy: 0.8034 - 1s/epoch - 6ms/step
Epoch 7/50
176/176 - 1s - loss: 0.4119 - accuracy: 0.8011 - 1s/epoch - 6ms/step
Epoch 8/50
176/176 - 1s - loss: 0.4097 - accuracy: 0.8050 - 1s/epoch - 6ms/step
Epoch 9/50
176/176 - 1s - loss: 0.4087 - accuracy: 0.8068 - 1s/epoch - 6ms/step
Epoch 10/50
176/176 - 1s - loss: 0.4075 - accuracy: 0.8080 - 1s/epoch - 7ms/step
Epoch 11/50
176/176 - 1s - loss: 0.4063 - accuracy: 0.8059 - 1s/epoch - 6ms/step
Epoch 12/50
176/176 - 1s - loss: 0.4060 - accuracy: 0.8080 - 1s/epoch - 6ms/step
Epoch 13/50
176/176 - 1s - loss: 0.4

In [26]:
print('\nClassification Report:\n')
print(classification_report(y_test, y_pred))


Classification Report:

              precision    recall  f1-score   support

           0       0.84      0.88      0.86      1033
           1       0.61      0.52      0.56       374

    accuracy                           0.79      1407
   macro avg       0.72      0.70      0.71      1407
weighted avg       0.78      0.79      0.78      1407



**class 1 has lower performance with lower precision, recall, and F1-score.**

# Imbalanced Dataset
reference: https://www.kaggle.com/rafjaa/resampling-strategies-for-imbalanced-datasets



## Method 1: Undersampling

In [27]:
# class count
count_class_0, count_class_1 = df2['Churn'].value_counts()

# Divide by class
df_class_0 = df2[df2['Churn'] == 0]
df_class_1 = df2[df2['Churn'] == 1]

In [28]:
# Undersample 0-class and concat the df of both class
df_class_0_under = df_class_0.sample(count_class_1)
df_undersample = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(df_undersample['Churn'].value_counts())

Random under-sampling:
Churn
0    1869
1    1869
Name: count, dtype: int64


In [29]:
X = df_undersample.drop('Churn', axis=1)
y = df_undersample['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [30]:
y_train.value_counts()

Churn
0    1495
1    1495
Name: count, dtype: int64

In [31]:
y_pred = ANN(X_train, X_test, y_train, y_test)

Epoch 1/50
94/94 - 1s - loss: 0.6351 - accuracy: 0.6452 - 1s/epoch - 14ms/step
Epoch 2/50
94/94 - 1s - loss: 0.5364 - accuracy: 0.7462 - 685ms/epoch - 7ms/step
Epoch 3/50
94/94 - 1s - loss: 0.5066 - accuracy: 0.7656 - 699ms/epoch - 7ms/step
Epoch 4/50
94/94 - 1s - loss: 0.4974 - accuracy: 0.7649 - 719ms/epoch - 8ms/step
Epoch 5/50
94/94 - 1s - loss: 0.4928 - accuracy: 0.7672 - 715ms/epoch - 8ms/step
Epoch 6/50
94/94 - 1s - loss: 0.4902 - accuracy: 0.7639 - 735ms/epoch - 8ms/step
Epoch 7/50
94/94 - 1s - loss: 0.4889 - accuracy: 0.7712 - 704ms/epoch - 7ms/step
Epoch 8/50
94/94 - 1s - loss: 0.4877 - accuracy: 0.7669 - 722ms/epoch - 8ms/step
Epoch 9/50
94/94 - 1s - loss: 0.4851 - accuracy: 0.7712 - 716ms/epoch - 8ms/step
Epoch 10/50
94/94 - 1s - loss: 0.4830 - accuracy: 0.7702 - 729ms/epoch - 8ms/step
Epoch 11/50
94/94 - 1s - loss: 0.4819 - accuracy: 0.7716 - 704ms/epoch - 7ms/step
Epoch 12/50
94/94 - 1s - loss: 0.4800 - accuracy: 0.7719 - 706ms/epoch - 8ms/step
Epoch 13/50
94/94 - 1s - lo

In [32]:
print('\nClassification Report:\n')
print(classification_report(y_test, y_pred))


Classification Report:

              precision    recall  f1-score   support

           0       0.78      0.70      0.73       374
           1       0.72      0.80      0.76       374

    accuracy                           0.75       748
   macro avg       0.75      0.75      0.75       748
weighted avg       0.75      0.75      0.75       748



**Results after Undersampling**
- f1-score for minority class 1 improved from **0.6 to 0.75**
- f1-score for class 0 reduced from **0.86 to 0.75**
- We have more generalized classifier which classifies both classes with similar prediction score

## Method 2: Oversampling

In [33]:
# Oversample class-1 and concat the df of both classes
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_oversample = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_oversample['Churn'].value_counts())

Random over-sampling:
Churn
0    5163
1    5163
Name: count, dtype: int64


In [34]:
X = df_oversample.drop('Churn', axis=1)
y = df_oversample['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [35]:
y_train.value_counts()

Churn
0    4130
1    4130
Name: count, dtype: int64

In [36]:
y_pred = ANN(X_train, X_test, y_train, y_test)

Epoch 1/50
259/259 - 2s - loss: 0.5981 - accuracy: 0.6770 - 2s/epoch - 9ms/step
Epoch 2/50
259/259 - 2s - loss: 0.5051 - accuracy: 0.7519 - 2s/epoch - 7ms/step
Epoch 3/50
259/259 - 2s - loss: 0.4926 - accuracy: 0.7571 - 2s/epoch - 7ms/step
Epoch 4/50
259/259 - 2s - loss: 0.4884 - accuracy: 0.7610 - 2s/epoch - 7ms/step
Epoch 5/50
259/259 - 2s - loss: 0.4858 - accuracy: 0.7626 - 2s/epoch - 7ms/step
Epoch 6/50
259/259 - 2s - loss: 0.4826 - accuracy: 0.7623 - 2s/epoch - 7ms/step
Epoch 7/50
259/259 - 2s - loss: 0.4803 - accuracy: 0.7649 - 2s/epoch - 7ms/step
Epoch 8/50
259/259 - 2s - loss: 0.4781 - accuracy: 0.7677 - 2s/epoch - 7ms/step
Epoch 9/50
259/259 - 2s - loss: 0.4755 - accuracy: 0.7699 - 2s/epoch - 7ms/step
Epoch 10/50
259/259 - 2s - loss: 0.4743 - accuracy: 0.7694 - 2s/epoch - 7ms/step
Epoch 11/50
259/259 - 2s - loss: 0.4725 - accuracy: 0.7685 - 2s/epoch - 7ms/step
Epoch 12/50
259/259 - 2s - loss: 0.4704 - accuracy: 0.7735 - 2s/epoch - 7ms/step
Epoch 13/50
259/259 - 2s - loss: 0.46

In [37]:
print('\nClassification Report:\n')
print(classification_report(y_test, y_pred))


Classification Report:

              precision    recall  f1-score   support

           0       0.83      0.71      0.77      1033
           1       0.75      0.85      0.80      1033

    accuracy                           0.78      2066
   macro avg       0.79      0.78      0.78      2066
weighted avg       0.79      0.78      0.78      2066



**Results after Oversampling**
| Method       | f1-score (class 1) | f1-score (class 0)|
|--------------|--------------------|-------------------|
| Initial      | 0.56               |0.86               |
| Undersampling| 0.76               |0.73               |
| Oversampling | 0.8                |0.77               |

We have more generalized classifier which classifies both classes with similar prediction score

## Method 3: SMOTE
SMOTE (Synthetic Minority Over-sampling Technique) is a resampling method commonly used in machine learning to address class imbalance. It generates synthetic instances of the minority class by interpolating between existing minority class instances, thereby balancing the class distribution and improving the performance of models on imbalanced datasets.

To install imbalanced-learn library

In [38]:
# !pip install imbalanced-learn

In [39]:
X = df2.drop('Churn',axis='columns')
y = df2['Churn']

In [40]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)

y_sm.value_counts()

Churn
0    5163
1    5163
Name: count, dtype: int64

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=42, stratify=y_sm)

In [42]:
y_pred = ANN(X_train, X_test, y_train, y_test)

Epoch 1/50
259/259 - 3s - loss: 0.5616 - accuracy: 0.7292 - 3s/epoch - 10ms/step
Epoch 2/50
259/259 - 2s - loss: 0.4637 - accuracy: 0.7835 - 2s/epoch - 7ms/step
Epoch 3/50
259/259 - 2s - loss: 0.4532 - accuracy: 0.7861 - 2s/epoch - 7ms/step
Epoch 4/50
259/259 - 2s - loss: 0.4495 - accuracy: 0.7870 - 2s/epoch - 7ms/step
Epoch 5/50
259/259 - 2s - loss: 0.4461 - accuracy: 0.7887 - 2s/epoch - 7ms/step
Epoch 6/50
259/259 - 2s - loss: 0.4422 - accuracy: 0.7891 - 2s/epoch - 7ms/step
Epoch 7/50
259/259 - 2s - loss: 0.4390 - accuracy: 0.7916 - 2s/epoch - 7ms/step
Epoch 8/50
259/259 - 2s - loss: 0.4368 - accuracy: 0.7924 - 2s/epoch - 7ms/step
Epoch 9/50
259/259 - 2s - loss: 0.4341 - accuracy: 0.7969 - 2s/epoch - 7ms/step
Epoch 10/50
259/259 - 2s - loss: 0.4304 - accuracy: 0.7987 - 2s/epoch - 7ms/step
Epoch 11/50
259/259 - 2s - loss: 0.4295 - accuracy: 0.7988 - 2s/epoch - 7ms/step
Epoch 12/50
259/259 - 2s - loss: 0.4266 - accuracy: 0.8021 - 2s/epoch - 7ms/step
Epoch 13/50
259/259 - 2s - loss: 0.4

In [43]:
print('\nClassification Report:\n')
print(classification_report(y_test, y_pred))


Classification Report:

              precision    recall  f1-score   support

           0       0.80      0.79      0.79      1033
           1       0.79      0.80      0.80      1033

    accuracy                           0.80      2066
   macro avg       0.80      0.80      0.80      2066
weighted avg       0.80      0.80      0.80      2066



**Results**
| Method       | f1-score (class 1) | f1-score (class 0)|Accuracy|
|--------------|--------------------|-------------------|--------|
| Initial      | 0.56               |0.86               |0.79    | 
| Undersampling| 0.76               |0.73               |0.75    |
| Oversampling | 0.8                |0.77               |0.78    | 
| SMOTE        | 0.8                |0.79               |0.80    | 
- SMOTE increases f1-score of minority class 1 from **0.56 to 0.80**
- f1-score of class 0 reduced from **0.86 to 0.79**, but that's ok.

## Method 4: Use of Ensemble with undersampling

In [44]:
df2['Churn'].value_counts()

Churn
0    5163
1    1869
Name: count, dtype: int64

In [45]:
X = df2.drop('Churn', axis=1)
y = df2['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

y_train.value_counts()

Churn
0    4130
1    1495
Name: count, dtype: int64

In [46]:
df3 = X_train.copy()
df3['Churn'] = y_train

df3_class_0 = df3[df3['Churn']==0]
df3_class_1 = df3[df3['Churn']==1]

In [47]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Churn', axis='columns')
    y_train = df_train.Churn
    return X_train, y_train

**Batches**
|model|class0|class1|
|-----|------|------|
|1|1495|(0, 1495)|
|2|1495|(1496, 2990)|
|3|1495|(2991, 4130)|


In [48]:
# 1st Batch
X_train, y_train = get_train_batch(df3_class_0, df3_class_1, 0, 1495)
y_pred1 = ANN(X_train, X_test, y_train, y_test)

# 2nd Batch
X_train, y_train = get_train_batch(df3_class_0, df3_class_1, 1496, 2990)
y_pred2 = ANN(X_train, X_test, y_train, y_test)

# 3rd Batch
X_train, y_train = get_train_batch(df3_class_0, df3_class_1, 2991, 4130)
y_pred3 = ANN(X_train, X_test, y_train, y_test)

Epoch 1/50
94/94 - 1s - loss: 0.6586 - accuracy: 0.6010 - 1s/epoch - 13ms/step
Epoch 2/50
94/94 - 1s - loss: 0.5453 - accuracy: 0.7274 - 652ms/epoch - 7ms/step
Epoch 3/50
94/94 - 1s - loss: 0.5113 - accuracy: 0.7492 - 639ms/epoch - 7ms/step
Epoch 4/50
94/94 - 1s - loss: 0.4954 - accuracy: 0.7565 - 651ms/epoch - 7ms/step
Epoch 5/50
94/94 - 1s - loss: 0.4883 - accuracy: 0.7559 - 656ms/epoch - 7ms/step
Epoch 6/50
94/94 - 1s - loss: 0.4837 - accuracy: 0.7629 - 652ms/epoch - 7ms/step
Epoch 7/50
94/94 - 1s - loss: 0.4809 - accuracy: 0.7656 - 646ms/epoch - 7ms/step
Epoch 8/50
94/94 - 1s - loss: 0.4778 - accuracy: 0.7666 - 636ms/epoch - 7ms/step
Epoch 9/50
94/94 - 1s - loss: 0.4761 - accuracy: 0.7632 - 637ms/epoch - 7ms/step
Epoch 10/50
94/94 - 1s - loss: 0.4744 - accuracy: 0.7649 - 631ms/epoch - 7ms/step
Epoch 11/50
94/94 - 1s - loss: 0.4721 - accuracy: 0.7696 - 646ms/epoch - 7ms/step
Epoch 12/50
94/94 - 1s - loss: 0.4704 - accuracy: 0.7702 - 653ms/epoch - 7ms/step
Epoch 13/50
94/94 - 1s - lo

In [49]:
y_pred_combined = y_pred1 + y_pred2 + y_pred3
#  considers a "majority vote" to make the final prediction.
y_pred_combined = (y_pred_combined>1).astype(int)
y_pred_combined.T

array([[0, 1, 0, ..., 0, 0, 0]])

In [50]:
print('classification_report')
print(classification_report(y_test, y_pred_combined))

classification_report
              precision    recall  f1-score   support

           0       0.92      0.68      0.78      1033
           1       0.48      0.83      0.61       374

    accuracy                           0.72      1407
   macro avg       0.70      0.75      0.70      1407
weighted avg       0.80      0.72      0.74      1407



This method doesn't seem to give better f1-score.